<a href="https://colab.research.google.com/github/DKwokAsc/MCMC-Final-Project/blob/main/Wisconsin_MCMC_GerryChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Wisconsin MCMC GerryChain Partition and Elections**

This file creates partitions and ensembles for Markov Chain, Partitions and Elections


---


Requirements:
- `wi_2024_gen_prec_graph.json` must already be downloaded and inputted in the default file folder. If not, see the instructions on `Wisconsin_SHP_File_Conversion.ipynb`





In [ ]:
# Install gerrychain packages
!pip install gerrychain[geo]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 2.6 MB/s eta 0:00:00


In [ ]:
# Install additional dependencies
# Credit to the GerryChain documentation & tutorial for providing many of the
# examples done in this demo
from gerrychain import Graph, Partition, MarkovChain, Election, metrics
from gerrychain.proposals import propose_random_flip,recom
from gerrychain.updaters import Tally, cut_edges
from gerrychain.constraints import Validator, single_flip_contiguous, contiguous, within_percent_of_ideal_population # used for validators
from gerrychain.accept import always_accept
from functools import partial

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import pickle

import random
random.seed(42)

#**Winconsin Election Addition**

In [ ]:
# Remove empty row warnings (to decide later)
import warnings
warnings.filterwarnings("ignore")

# Initialize the graph for chains
graph = Graph.from_json("./wi_2024_gen_prec_graph.json")

# Create the election first to validate data
election = Election(
    "PRES24",
    {"Dem":"PREDEM24", "Rep":"PREREP24"}
)

# Create 3 plans: One for congressionalm one for state assembly, one for state district

cong_partition = Partition(
    graph,
    assignment="CONG_DIST",
    updaters={
        "population": Tally("PERSONS", alias="population"),
        "cut_edges": cut_edges,
        "PRES24": election
    }
)

state_assem_partition = Partition(
    graph,
    assignment="SLDL_DIST",
    updaters={
        "population": Tally("PERSONS", alias="population"),
        "cut_edges": cut_edges,
        "PRES24": election
    }
)

state_dist_partition = Partition(
    graph,
    assignment="SLDU_DIST",
    updaters={
        "population": Tally("PERSONS", alias="population"),
        "cut_edges": cut_edges,
        "PRES24": election
    }
)

# Prints output returns population info based on the plan chosen
def print_population(name, partition):
  print(f"\n{name} Plan Population:")
  for district, pop in partition["population"].items():
      print(f"  District {district}: {pop}")


# Sanity check
print_population("Congressional", cong_partition)
print_population("State Assembly", state_assem_partition)
print_population("State district", state_dist_partition)


Congressional Plan Population:
  District 03: 736716.0
  District 07: 736715.0
  District 08: 736714.0
  District 06: 736692.0
  District 02: 736715.0
  District 05: 736737.0
  District 01: 736715.0
  District 04: 736714.0

State Assembly Plan Population:
  District 72: 59742.0
  District 57: 59642.0
  District 41: 59672.0
  District 39: 59796.0
  District 71: 59463.0
  District 74: 58968.0
  District 73: 59853.0
  District 67: 60062.0
  District 88: 59855.0
  District 89: 59697.0
  District 02: 59983.0
  District 01: 59444.0
  District 90: 59551.0
  District 04: 60096.0
  District 05: 59076.0
  District 06: 59188.0
  District 29: 59983.0
  District 75: 60058.0
  District 03: 59173.0
  District 27: 60058.0
  District 68: 59902.0
  District 69: 59952.0
  District 91: 60072.0
  District 92: 58940.0
  District 42: 59201.0
  District 40: 59478.0
  District 37: 59609.0
  District 49: 59584.0
  District 47: 58987.0
  District 46: 59231.0
  District 50: 59024.0
  District 81: 60040.0
  Distr

**Efficiency Gap measuring for democratic and republican party**

In [ ]:

# Function that calculate results based on the selected partition
# Returns: two outputs; the efficiency gap of the selected parition and the number of seats won by republicans
def summarize_election(partition):
  # Create the election based on the selected map "partition"
  election_map = partition["PRES24"]
  # Compute the efficiency gap using metrics.efficiency_gap
  efficiency_gap = metrics.efficiency_gap(election_map)

  # Compute the number of districts won by republicans
  rep_wins = election_map.seats("Rep")

  return efficiency_gap, rep_wins

# Function: Printing function for election results
# Returns: Print desired output based on the selected map, its efficiency gap and seats taken
def print_results(name, efficiency_gap, districts_won):
  print(f"{name} Voting Results")
  print(f"Efficiency gap: {efficiency_gap:0.4f}")
  print(f"Districts won: {districts_won}")

eg_dem, rep_wins_dem = summarize_election(cong_partition)
print_results("Congressional Plan", eg_dem, rep_wins_dem)

eg_rep, rep_wins_rep = summarize_election(state_assem_partition)
print_results("State Assembly Plan", eg_rep, rep_wins_rep)

eg_comp, rep_wins_comp = summarize_election(state_dist_partition)
print_results("State District Plan", eg_rep, rep_wins_comp)

Congressional Plan Voting Results
Efficiency gap: -0.2535
Districts won: 6
State Assembly Plan Voting Results
Efficiency gap: -0.0261
Districts won: 50
State District Plan Voting Results
Efficiency gap: -0.0261
Districts won: 15


#**Create Mini Ensembles for all 3 Partitions**

In [ ]:
# Number of plans
num_plans = 100
cong_ensemble = []

# Create the Markov chain
chain = MarkovChain(
    proposal= propose_random_flip,
    accept=always_accept,
    initial_state=cong_partition,
    total_steps=num_plans
)

# Generate plans
for step, partition in enumerate(chain):
    cong_ensemble.append(partition)
print(f"Generated {len(cong_ensemble)} plans.")

Generated 100 plans.


In [ ]:
# Use the same partition from the all favoring plans (In first cell already)

# Intialize the target population
# We need total population (dictionnary) and the number of districts (list) to find ideal population for a single district
tot_pop = sum(cong_partition["population"].values())
print(tot_pop)

tot_districts = len(cong_partition.parts)

target_pop = tot_pop/tot_districts
print(target_pop)

# Add the ReCom proposal method
proposal = partial(
    recom,
    pop_col="PERSONS",
    pop_target=target_pop,
    epsilon=0.01,
    node_repeats=2
)

# Make every plan follow these constraints

constraints = [within_percent_of_ideal_population(cong_partition, 0.01)]
# Add the Markov Chain
chain = MarkovChain(proposal=proposal, constraints=constraints,accept=always_accept, initial_state=cong_partition, total_steps=100)

# Sanity check
for (i,part) in enumerate(chain):
  print(f"Step {i} Rep vote share for dist 1: "
    f"{part['PRES24'].percents('Rep')[1]:0.4f}")


5893718.0
736714.75
Step 0 Rep vote share for dist 1: 0.6142


KeyboardInterrupt: 